In [29]:
!pip install scikit-learn pandas

In [28]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [34]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier
from sklearn.impute import SimpleImputer
from sklearn.metrics import accuracy_score, confusion_matrix, classification_report
from sklearn.preprocessing import LabelEncoder

In [42]:
train_data = pd.read_csv("/content/drive/MyDrive/train_data.csv")
train_label = pd.read_csv("/content/drive/MyDrive/train_label.csv")
test_data  = pd.read_csv("/content/drive/MyDrive/test_data.csv")
test_label = pd.read_csv("/content/drive/MyDrive/test_label.csv")

df_train = train_data.merge(train_label, on="id", how="inner")

X_train = df_train.drop(columns=["id", "Class"]).values
y_train_text = df_train["Class"].values

ids_test = test_data["id"].values
X_test  = test_data.drop(columns=["id"]).values

imputer = SimpleImputer(missing_values=np.nan, strategy="mean")

X_train_imputed = imputer.fit_transform(X_train)
X_test_imputed  = imputer.transform(X_test)
le = LabelEncoder()
y_train = le.fit_transform(y_train_text)

rf = RandomForestClassifier(
    n_estimators=100,
    random_state=42,
    n_jobs=-1
)
rf.fit(X_train_imputed, y_train)

proba = rf.predict_proba(X_test_imputed)
y_pred_int = np.argmax(proba, axis=1)
confidence_scores = np.max(proba, axis=1)
y_pred_text = le.inverse_transform(y_pred_int)
threshold = 0.75
temp_df = pd.DataFrame({
    "id": ids_test,
    "Class": y_pred_text,
    "confidence": confidence_scores
})

known_ids   = temp_df.loc[temp_df["confidence"] >= threshold, "id"].values
unknown_ids = temp_df.loc[temp_df["confidence"] <  threshold, "id"].values
known_df_label = temp_df.loc[temp_df["id"].isin(known_ids), ["id", "Class"]]
known_df_label.to_csv("/content/drive/MyDrive/RF_known.csv", index=False)
unknown_full = test_data.loc[test_data["id"].isin(unknown_ids)].copy()
unknown_full.to_csv("/content/drive/MyDrive/RF_unknown.csv", index=False)

113
219
